# Logistic Regression

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from logistic_regression import MultinomialLogisticRegression
from preprocess import Preprocess
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

### Importind data

In [3]:
# Paths of the raw data
# Plase changhe this in order to execute the code on your machine
raw_train_data = "../data/train.csv"
raw_test_data = "../data/test.csv"

In [16]:
# Preprocessing
preproc = Preprocess()
raw_data = preproc.load_data(raw_train_data)

# Select columns and preprocess
cols = ['lat', 'TS', 'Z1000', 'U850', 'PS', 'PSL', 'TMQ', 'Z200', 'lon', 'T500',
        "Label"] 

train_df, train_data = preproc.preprocess_data(raw_data, drop_cols=["SNo", "time"])
train_df = train_df[cols]

np.random.shuffle(train_data)
X_train, y_train, X_valid, y_valid = preproc.train_valid_split(
    train_df.to_numpy(), test_size=0.33 #, random_state=42
)


# Check columns
print(train_df.columns)

# Normalize data
X_train = preproc.normalize_data(X_train)
X_valid = preproc.normalize_data(X_valid)

Index(['lat', 'TS', 'Z1000', 'U850', 'PS', 'PSL', 'TMQ', 'Z200', 'lon', 'T500',
       'Label'],
      dtype='object')


Removing outliers

In [13]:
# Removing the rows that pass the threshold
threshold = 4
X_train = X_train[
    (np.abs(X_train[:,2]) < threshold) 
    & (np.abs(X_train[:,3]) < threshold)
    & (np.abs(X_train[:,4]) < threshold)
    & (np.abs(X_train[:,5]) < threshold)
    & (np.abs(X_train[:,6]) < threshold)
    & (np.abs(X_train[:,7]) < threshold)
    & (np.abs(X_train[:,8]) < threshold)
    & (np.abs(X_train[:,9]) < threshold),
    :
]

### Training model

In [21]:
model = MultinomialLogisticRegression(learning_rate=0.1, num_iterations=1000, regularizer=0.05)
model.fit(X_train, y_train, collist=raw_data.columns, valid_x=X_valid, valid_y=y_valid)

# Make predictions.
predictions = model.predict(X_valid)
print(train_df.columns)

model.get_metrics(y_valid, predictions, return_values=False)

Epoch 0, cross entropy loss: 1.0986122886681096
Epoch 100, cross entropy loss: 1.3319577682487105
Epoch 200, cross entropy loss: 1.3522162466581074
Epoch 300, cross entropy loss: 1.3440761903632488
Epoch 400, cross entropy loss: 1.3380537770247012
Epoch 500, cross entropy loss: 1.336133265179013
Epoch 600, cross entropy loss: 1.3364211734399578
Epoch 700, cross entropy loss: 1.3375626636866498
Epoch 800, cross entropy loss: 1.3389168156016933
Epoch 900, cross entropy loss: 1.3402252040101712
Index(['T200', 'Z200', 'VBOT', 'UBOT', 'T500', 'U850', 'TS', 'TMQ', 'V850',
       'Label'],
      dtype='object')
Confusion Matrix:
[[3534 5259 2836]
 [  48  482   48]
 [ 226  667 1671]] 

Accuracy:
0.385 

Precision:
0.4567 

Recall:
0.5965 

F1 Score:
0.5173


This is really bad result.